In [1]:
#### Helpers ####
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import settings

bucket = S3BucketUtils()
################

import pandas as pd
date_of_analysis = input('Date of analysis: ')

Date of analysis: 2022-02-01


In [2]:
def did_before(df, export, metric_name, var_name, spots_set, with_wo_CB, event_date_full_name):
    closed_sold_business_spots = df[df['Reason for Cancelling Survey'].\
                apply(lambda x: 'Sold' in x or 'Closed' in x)]['spot_id'].unique()
    
    export.loc[metric_name, spots_set+'_spots_'+with_wo_CB+'_'+event_date_full_name] = \
    (df[(df[var_name]==1)]['spot_id'].nunique(),\
     round(100*(df[df[var_name]==1]['spot_id'].nunique()/df['spot_id'].nunique()), 2))
    
    return export

def did_before_and_churned_at_some_point(df, export, metric_name, var_name, spots_set, with_wo_CB, event_date_full_name):
    closed_sold_business_spots = df[df['Reason for Cancelling Survey'].\
                apply(lambda x: 'Sold' in x or 'Closed' in x)]['spot_id'].unique()
    
    export.loc[metric_name, spots_set+'_spots_'+with_wo_CB+'_'+event_date_full_name] = \
    (df[(df[var_name]==1)&\
       (df['spot_id'].isin(df[df['event']==True]['spot_id'].unique()))]['spot_id'].nunique(),\
     round(100*(df[(df[var_name]==1)&\
       (df['spot_id'].isin(df[df['event']==True]['spot_id'].unique()))]['spot_id'].nunique()/df[df[var_name]==1]['spot_id'].nunique()), 2))
    
    return export

def did_before_and_churned_at_some_point_ClosedSoldBusiness(df, export, metric_name, var_name, spots_set, with_wo_CB, event_date_full_name):
    closed_sold_business_spots = df[df['Reason for Cancelling Survey'].\
                apply(lambda x: 'Sold' in x or 'Closed' in x)]['spot_id'].unique()
    
    export.loc[metric_name, spots_set+'_spots_'+with_wo_CB+'_'+event_date_full_name] = \
    (df[(df[var_name]==1)&\
       (df['spot_id'].isin(df[df['event']==True]['spot_id'].unique()))&\
       (df['spot_id'].isin(closed_sold_business_spots))]['spot_id'].nunique(),\
     round(100*(df[(df[var_name]==1)&\
       (df['spot_id'].isin(df[df['event']==True]['spot_id'].unique()))&\
                  (df['spot_id'].isin(closed_sold_business_spots))]['spot_id'].nunique()/df[df[var_name]==1]['spot_id'].nunique()), 2))
    
    return export

def did_before_and_churned_the_next_month(df, export, metric_name, var_name, spots_set, with_wo_CB, event_date_full_name):
    closed_sold_business_spots = df[df['Reason for Cancelling Survey'].\
                apply(lambda x: 'Sold' in x or 'Closed' in x)]['spot_id'].unique()
    
    export.loc[metric_name, spots_set+'_spots_'+with_wo_CB+'_'+event_date_full_name] = \
    (df[(df[var_name]==1)&\
       (df['event']==True)]['spot_id'].nunique(),\
     round(100*(df[(df[var_name]==1)&\
       (df['event']==True)]['spot_id'].nunique()/df[df[var_name]==1]['spot_id'].nunique()), 2))
    
    return export

def did_before_and_churned_the_next_month_ClosedSoldBusiness(df, export, metric_name, var_name, spots_set, with_wo_CB, event_date_full_name):
    closed_sold_business_spots = df[df['Reason for Cancelling Survey'].\
                apply(lambda x: 'Sold' in x or 'Closed' in x)]['spot_id'].unique()
    
    export.loc[metric_name, spots_set+'_spots_'+with_wo_CB+'_'+event_date_full_name] = \
    (df[(df[var_name]==1)&\
       (df['event']==True)&\
       (df['spot_id'].isin(closed_sold_business_spots))]['spot_id'].nunique(),\
     round(100*(df[(df[var_name]==1)&\
       (df['event']==True)&\
                  ((df['spot_id'].isin(closed_sold_business_spots)))]['spot_id'].nunique()/df[df[var_name]==1]['spot_id'].nunique()), 2))
    
    return export

In [3]:
def main(date_of_analysis, spots_sets, with_wo_CB_options, event_date_full_names):
    date_dir = date_of_analysis.replace('-', '_')
    
    did_something_before_var = 'had_clicked_emails_before'
    did_something_last_X_months_var = 'had_clicked_emails_last_4_months'
    did_something_before_and_didnt_last_X_months_var = 'had_clicked_emails_before_and_didnt_last_4_months'
    
    all_spots_sets = []
    for spots_set in spots_sets:
        for with_wo_CB in with_wo_CB_options:
            for event_date_full_name in event_date_full_names:
                all_spots_sets.append(spots_set+'_spots_'+with_wo_CB+'_'+event_date_full_name)
                
    export = pd.DataFrame(index = ['all_spots',\
                              'did_something_before',\
                              'did_something_before_and_churned_at_some_point',\
                              'did_something_before_and_churned_at_some_point_Closed/Sold_Business',\
                              'did_something_before_and_churned_the_next_month',\
                              'did_something_before_and_churned_the_next_month_Closed/Sold_Business',\
                              'did_something_last_X_months',\
                              'did_something_last_X_months_and_churned_at_some_point',\
                               'did_something_last_X_months_and_churned_at_some_point_Closed/Sold_Business',\
                              'did_something_last_X_months_and_churned_the_next_month',\
                               'did_something_last_X_months_and_churned_the_next_month_Closed/Sold_Business',\
                              'did_something_before_and_didnt_last_X_months',\
                              'did_something_before_and_didnt_last_X_months_and_churned_at_some_point',\
                               'did_something_before_and_didnt_last_X_months_and_churned_at_some_point_Closed/Sold_Business',\
                              'did_something_before_and_didnt_last_X_months_and_churned_the_next_month',\
                              'did_something_before_and_didnt_last_X_months_and_churned_the_next_month_Closed/Sold_Business'],\
                     columns = all_spots_sets)
    
    for spots_set in spots_sets:
        for with_wo_CB in with_wo_CB_options:
            for event_date_full_name in event_date_full_names:
                if event_date_full_name == 'cancellation_requested':
                    event_date = 'canc_req'
                elif event_date_full_name == 'cancellation_confirmed':
                    event_date = 'canc_conf'
                    
                df = bucket.load_csv_from_s3(file_name = 'churn_analysis_based_on_behaviour/data/' + date_dir + \
                '/exports/data_used_for_each_model/data_tv_'+spots_set+'_spots_'+with_wo_CB+'_'+event_date+'.csv')

                df['Reason for Cancelling Survey'].fillna('no reason', inplace = True)
                    
                
                export.loc['all_spots', spots_set+'_spots_'+with_wo_CB+'_'+event_date_full_name] = \
                df['spot_id'].nunique()
                
                
                export = did_before(df=df, export=export, metric_name='did_something_before', \
                    var_name=did_something_before_var, spots_set=spots_set,\
                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_at_some_point(df=df, export=export, metric_name='did_something_before_and_churned_at_some_point', \
                                    var_name=did_something_before_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_at_some_point_ClosedSoldBusiness(df=df, export=export, metric_name='did_something_before_and_churned_at_some_point_Closed/Sold_Business', \
                                    var_name=did_something_before_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_the_next_month(df=df, export=export, metric_name='did_something_before_and_churned_the_next_month', \
                                    var_name=did_something_before_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_the_next_month_ClosedSoldBusiness(df=df, export=export, metric_name='did_something_before_and_churned_the_next_month_Closed/Sold_Business', \
                                    var_name=did_something_before_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)

                export = did_before(df=df, export=export, metric_name='did_something_last_X_months', \
                                    var_name=did_something_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_at_some_point(df=df, export=export, metric_name='did_something_last_X_months_and_churned_at_some_point', \
                                    var_name=did_something_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_at_some_point_ClosedSoldBusiness(df=df, export=export, metric_name='did_something_last_X_months_and_churned_at_some_point_Closed/Sold_Business', \
                                    var_name=did_something_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_the_next_month(df=df, export=export, metric_name='did_something_last_X_months_and_churned_the_next_month', \
                                    var_name=did_something_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_the_next_month_ClosedSoldBusiness(df=df, export=export, metric_name='did_something_last_X_months_and_churned_the_next_month_Closed/Sold_Business', \
                                    var_name=did_something_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)

                export = did_before(df=df, export=export, metric_name='did_something_before_and_didnt_last_X_months', \
                                    var_name=did_something_before_and_didnt_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_at_some_point(df=df, export=export, metric_name='did_something_before_and_didnt_last_X_months_and_churned_at_some_point', \
                                    var_name=did_something_before_and_didnt_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_at_some_point_ClosedSoldBusiness(df=df, export=export, metric_name='did_something_before_and_didnt_last_X_months_and_churned_at_some_point_Closed/Sold_Business', \
                                    var_name=did_something_before_and_didnt_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_the_next_month(df=df, export=export, metric_name='did_something_before_and_didnt_last_X_months_and_churned_the_next_month', \
                                    var_name=did_something_before_and_didnt_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                export = did_before_and_churned_the_next_month_ClosedSoldBusiness(df=df, export=export, metric_name='did_something_before_and_didnt_last_X_months_and_churned_the_next_month_Closed/Sold_Business', \
                                    var_name=did_something_before_and_didnt_last_X_months_var, spots_set=spots_set,\
                                    with_wo_CB=with_wo_CB, event_date_full_name=event_date_full_name)
                
                export.to_csv('data/'+date_dir+'/exports/had_clicked_emails_stats.csv')
                
    
    return export

In [4]:
export = main(date_of_analysis=date_of_analysis,\
              spots_sets=['ALL', 'CAN_CANCEL'],\
              with_wo_CB_options=['with_CB', 'wo_CB'],\
              event_date_full_names=['cancellation_confirmed', 'cancellation_requested'])